In [1]:
import random
import numpy as np
from generate import data

import warnings
warnings.filterwarnings("ignore")

### Non-sparse setting (Optional)

In [2]:
# 特征维度
p = 1000
mean1 = np.repeat(0,p)
mean2 = np.repeat(1,p)
mean0 = np.repeat(0,p)
mean = [mean1,mean2,mean0]

# 方差矩阵列表
cov1 = np.diag(np.repeat(1,p))
cov2 = np.diag(np.repeat(1,p))
cov0 = np.diag(np.repeat(3.5,p))
cov = [cov1,cov2,cov0]

# 样本数量

n_train = 510
n_test = 2000

# 测试集中内点比例
purity = 2/3

# 是否添加均匀分布噪声
mixture = True

### Sparse Setting (Optional)

In [ ]:
""" # 特征维度
p = 1000
d = 500
mean1 = np.repeat(0,p)
mean2 = np.concatenate((np.repeat(1,d),np.repeat(0,p-d))) 
mean0 = np.concatenate((np.repeat(-1,d),np.repeat(0,p-d)))
mean = [mean1,mean2,mean0]

# 方差矩阵列表
cov1 = np.diag(np.repeat(1,p))
cov2 = np.diag(np.concatenate((np.array([1,1]),np.repeat(1, p-2))))
cov0 = np.diag(np.concatenate((np.repeat(1,d), np.repeat(1, p-d))))
for cov in [cov0,cov1,cov2]:
    for i in range(p):
        for j in range(p):
            if i!=j:
                cov[i,j] = 0.8
cov = [cov1,cov2,cov0]

# 样本数量

n_train = 510
n_test = 2000

# 测试集中内点比例
purity = 2/3

# 是否添加均匀分布噪声
mixture = True """

### Classification

In [3]:
from RPCP import RPEnsemble_CP
from Base_classify import base
from Aggregate import AggregatedCP

In [4]:
n_exp = 5
exp_model = "RandomForest"
alpha = 0.05
base_acc = []
rpecp_acc, rpecp_coverage, rpecp_length = [], [], [] 
icp_acc, icp_coverage, icp_length = [], [], []
ccp_acc, ccp_coverage, ccp_length = [], [], [] 
bcp_acc, bcp_coverage, bcp_length = [], [], [] 

for exp in range(n_exp):
    print("exp:", exp+1, "start:")
    X = data(mean, cov, mixture = mixture)
    X_train = X.generate(n_train, purity, shuffle=False, random_state = 2024+exp)
    X_test = X.generate(n_test, purity, shuffle=True, random_state = 2024+exp)

    rpecp = RPEnsemble_CP(rp="Gaussian", d = 200, B1=10, B2=50, classifier=exp_model, alpha=alpha)
    rpecp_res = rpecp.RP_Conformal(X_train, X_test)
    rpecp_acc.append(rpecp_res[0])
    rpecp_coverage.append(rpecp_res[1])
    rpecp_length.append(rpecp_res[2])
    print(rpecp_res)
    print("RPECP completed")

    base_classifier = base(classifier = exp_model)
    base_res = base_classifier.predict(X_train, X_test)
    base_acc.append(base_res)
    print(base_res)
    print("Base completed")

    ICPredictor = AggregatedCP(classifier=exp_model, alpha=alpha, mode="ICP")
    icp_res = ICPredictor.predict(X_train, X_test)
    icp_acc.append(icp_res[0])
    icp_coverage.append(icp_res[1])
    icp_length.append(icp_res[2])
    print("ICP completed")


    CCPredictor = AggregatedCP(classifier=exp_model, alpha=alpha, mode="CCP")
    ccp_res = CCPredictor.predict(X_train, X_test)
    ccp_acc.append(ccp_res[0])
    ccp_coverage.append(ccp_res[1])
    ccp_length.append(ccp_res[2])
    print("CCP completed")

    BCPredictor = AggregatedCP(classifier=exp_model, alpha=alpha, mode="BCP")
    bcp_res = BCPredictor.predict(X_train, X_test)
    bcp_acc.append(bcp_res[0])
    bcp_coverage.append(bcp_res[1])
    bcp_length.append(bcp_res[2])
    print("BCP completed")

    print("---------------------------------------\n")

exp: 1 start:
(0.92, 1.0, 1.8445)
RPECP completed
0.843
Base completed
ICP completed
CCP with 2 folds
CCP completed
BCP completed
---------------------------------------

exp: 2 start:
(0.9135, 1.0, 1.867)
RPECP completed
0.8445
Base completed
ICP completed
CCP with 2 folds
CCP completed
BCP completed
---------------------------------------

exp: 3 start:
(0.9305, 1.0, 1.868)
RPECP completed
0.841
Base completed
ICP completed
CCP with 2 folds
CCP completed
BCP completed
---------------------------------------

exp: 4 start:
(0.93, 1.0, 1.8685)
RPECP completed
0.844
Base completed
ICP completed
CCP with 2 folds
CCP completed
BCP completed
---------------------------------------

exp: 5 start:
(0.93, 1.0, 1.846)
RPECP completed
0.8505
Base completed
ICP completed
CCP with 2 folds
CCP completed
BCP completed
---------------------------------------



In [5]:
print("base acc:", np.mean(base_acc), np.std(base_acc))
print("-----------------------------------------------------")
print("RPECP acc:", np.mean(rpecp_acc), np.std(rpecp_acc))
print("RPECP coverage:", np.mean(rpecp_coverage), np.std(rpecp_coverage))
print("RPECP length:", np.mean(rpecp_length), np.std(rpecp_length))
print("-----------------------------------------------------")
print("ICP acc:", np.mean(icp_acc), np.std(icp_acc))
print("ICP coverage:", np.mean(icp_coverage), np.std(icp_coverage))
print("ICP length:", np.mean(icp_length), np.std(icp_length))
print("-----------------------------------------------------")
print("CCP acc:", np.mean(ccp_acc), np.std(ccp_acc))
print("CCP coverage:", np.mean(ccp_coverage), np.std(ccp_coverage))
print("CCP length:", np.mean(ccp_length), np.std(ccp_length))
print("-----------------------------------------------------")
print("BCP acc:", np.mean(bcp_acc), np.std(bcp_acc))
print("BCP coverage:", np.mean(bcp_coverage), np.std(bcp_coverage))
print("BCP length:", np.mean(bcp_length), np.std(bcp_length))

base acc: 0.8446 0.0031843366656181546
-----------------------------------------------------
RPECP acc: 0.9248 0.0068891218598599495
RPECP coverage: 1.0 0.0
RPECP length: 1.8588 0.011084223021935275
-----------------------------------------------------
ICP acc: 0.0103 0.004057092555020158
ICP coverage: 1.0 0.0
ICP length: 2.8164 0.025096613317338323
-----------------------------------------------------
CCP acc: 0.001 0.00044721359549995795
CCP coverage: 1.0 0.0
CCP length: 2.9414 0.006406246951218736
-----------------------------------------------------
BCP acc: 0.0016 0.00066332495807108
BCP coverage: 1.0 0.0
BCP length: 2.9822999999999995 0.0038678159211627417
